In [ ]:
from gears import PertData, GEARS

# get data
pert_data = PertData('./data')
# pert_data = PertData('./data_folder')
# load dataset in paper: norman, adamson, dixit.
pert_data.load(data_name = 'dixit')
# specify data split
pert_data.prepare_split(split = 'simulation', seed = 1)
# get dataloader with batch size
pert_data.get_dataloader(batch_size = 32, test_batch_size = 128)

from sklearn.model_selection import train_test_split
from gears.inference import evaluate, compute_metrics, deeper_analysis, non_dropout_analysis
from gears.utils import create_cell_graph_dataset_for_prediction

In [ ]:
import gears 
import pytorch_lightning
import seaborn as sns
sns.set_style("whitegrid")
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
%matplotlib inline
import pickle
import scanpy as sc
# import sentence_transformers
plt.style.use('ggplot')
#plt.style.use('seaborn-v0_8-dark-palette')
plt.rcParams['axes.facecolor'] = 'white'

np.random.seed(202310)
pytorch_lightning.seed_everything(202310)

In [ ]:
with open("/gpfs/gibbs/pi/zhao/tl688/GenePT/data_embedding/GPT_3_5_gene_embeddings.pickle", "rb") as fp:
    GPT_3_5_gene_embeddings = pickle.load(fp)
with open("/gpfs/gibbs/pi/zhao/tl688/cpsc_finalproject/genept_data/GenePT/ensem_emb_gpt3.5all.pickle", "rb") as fp:
    GPT_3_5_gene_embeddings = pickle.load(fp)
gene_names= list(pert_data.adata.var['gene_name'].values)
count_missing = 0
EMBED_DIM = 1536 # embedding dim from GPT-3.5
lookup_embed = np.zeros(shape=(len(gene_names),EMBED_DIM))
for i, gene in enumerate(gene_names):
    if gene in GPT_3_5_gene_embeddings:
        lookup_embed[i,:] = GPT_3_5_gene_embeddings[gene].flatten()
    else:
        count_missing+=1


In [ ]:
# set up and train a model
gears_model = GEARS(pert_data, device = 'cuda:0', gene_emb = lookup_embed)
# gears_model = GEARS(pert_data, device = 'cuda:0')
gears_model.model_initialize(hidden_size = 64)

gears_model.train(epochs = 20)

# # save/load model
# gears_model.save_model('gears_dixit_new')
# gears_model.load_pretrained('gears_dixit_new')

test_res = evaluate(gears_model.dataloader['test_loader'], gears_model.model, gears_model.config['uncertainty'], gears_model.device)
test_metrics, test_pert_res = compute_metrics(test_res)